In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.Experiments import execute_circuits

%load_ext autoreload
%autoreload 2

## Demonstrating error correction

In [ ]:
# Runs a shot on a noiseless simulator with an error at each of the physical qubits
shots = 1
simulator = AerSimulator()

outputs = []
for i in range(7):
    # Generates the circuit
    lqcirc = LogicalCircuit(2, **steane_code, name="SteaneCodeCircuit")    
    lqcirc.encode(0,1, max_iterations=1, initial_states=[0,1])

    # Inserts an error on both logical qubits
    lqcirc.add_error(0, i, 'X') # Can test X or Z errors
    lqcirc.add_error(1, i, 'X')

    # Adds QEC cycle and measurement
    lqcirc.append_qec_cycle([0, 1])
    lqcirc.measure([0, 1], [0, 1])

    # Simulates the QEC cycle
    result = execute_circuits(lqcirc, backend=simulator, shots=shots, memory=True)[0]
    output = result.get_memory(lqcirc)
    outputs.append(output[0][:2])

# Outputs should all be 10 if the error correction worked properly
print(outputs)

## Benchmarking encoding verification for SPAM

In [ ]:
from qiskit.visualization import plot_histogram

# Tests SPAM. Accuracy (historically) should be about 99.3%
initial_state = 0
max_max_it = 5
shots = int(1E3)

backend = "aer_simulator"

fig, ax = plt.subplots()
plot_option = "probs"
width = 0.3

outcomes = range(1, max_max_it+1)
all_data = []

for max_it in range(1, max_max_it+1):
    lqcirc = LogicalCircuit(1, **steane_code, name="test_circuit")
    lqcirc.encode(0, initial_states=[initial_state], max_iterations=max_it)
    lqcirc.measure([0], [0], with_error_correction=False)
    
    result = execute_circuits(lqcirc, backend=backend, hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None, shots=shots, memory=True)[0]
    output = result.get_memory(lqcirc)
    counts = lqcirc.get_logical_counts(output)

    if plot_option == "counts":
        data = counts
    elif plot_option == "probs":
        p0 = counts["0"]/shots if "0" in counts else 1-(counts["1"]/shots)
        data = [p0, 1-p0]
    else:
        raise ValueError(f"'{plot_option}' is not a valid plot_option; choose from ['counts', 'probs']")

    bar = ax.bar(max_it, data[0], width, log=False, label=f"max_iterations={max_it}")
    ax.bar_label(bar)

ax.set_xticks(outcomes)

ax.set_yscale("log")

ax.set_title("Encoding verification test")
ax.set_xlabel("Max number of iterations")
ax.set_ylabel(f"N({initial_state})" if plot_option == "counts" else f"P({initial_state})")

ax.legend()

plt.show()

## Comparing Hadamard gate implementations

In [ ]:
initial_state = 0
n_gates = 10
increments = 2   # Test every (increment) number of H gates
shots = 10000

lqcirc = LogicalCircuit(2, **steane_code, name="SteaneCodeCircuit")    
lqcirc.encode([0,1], max_iterations=1, initial_states=[0,1])

backend = "aer_simulator"

all_data = []

for max_it in range(n_gates + 1):
    # Create and encode logical circuit
    lqcirc = LogicalCircuit(1, **steane_code, name="test_circuit")
    lqcirc.encode(0, initial_states=[initial_state])

    # Append H gates max_it times on logical qubit 0
    for _ in range(max_it * increments):   # Add a +1 in the range constructor to test odd numbers of gates
        lqcirc.h(0)

    # Measure 
    lqcirc.measure([0], [0], with_error_correction=False)

    # Simulate
    result = execute_circuits(lqcirc, backend=backend, hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None, shots=shots, memory=True)[0]
    output = result.get_memory(lqcirc)
    all_data.append(output)

In [ ]:
accuracies = []
for output in all_data:
    log_counts = lqcirc.get_logical_counts(output)
    acc = log_counts['0'] / (log_counts['0'] + log_counts['1'])
    accuracies.append(acc)

h_gates_applied = range(0, (n_gates + 1) * increments, increments)
plt.plot(h_gates_applied, accuracies, 'o--')
plt.xlabel('Number of H gates applied')
plt.ylabel('Rate of Measured 0')
plt.show()

## Appending QEC cycles on demand

In [ ]:
lqcirc = LogicalCircuit(1, **steane_code)
lqcirc.encode(0, initial_states=[1])
lqcirc.append_qec_cycle([0])
#lqcirc.draw('mpl')

## Constructing LogicalCircuits from physical QuantumCircuits

In [ ]:
# We can also construct a logical circuit from a physical qubit circuit, first by constructing a QuantumCircuit:
pqc = QuantumCircuit(2)
pqc.x(0)
# pqc.h(0)
# pqc.cx(0,1)
pqc.draw(output="mpl")

In [ ]:
# Then, we convert the QuantumCircuit to a LogicalCircuit:
lqc = LogicalCircuit.from_physical_circuit(pqc, **steane_code)
lqc.draw(output="mpl")